In [1]:
using ADCME
using Plots
using PyCall
#### Read data
py"""
import numpy as np
"""

┌ Warning: Cannot load /home/darve/adncat/.julia/packages/ADCME/OGm7w/deps/CustomOps/build/libadcme.so. Please recompile the shared library by `ADCME.precompile()` for using custom operators.
└ @ ADCME /home/darve/adncat/.julia/packages/ADCME/OGm7w/src/ADCME.jl:76


In [2]:
function f_hi(a, T, R)
        v1 = (a[:,2] + a[:,3] * T + a[:,4] * T^2 + a[:,5] * T^3 + a[:,6] * T^4) * R
        v2 = (a[:,2] + a[:,3] * T / 2 + a[:,4] * T^2 /3 + a[:,5] * T^3 /4 + a[:,6] * T^4 /5 + a[:,7]/T) * R * T
        v3 = (a[:,2] * log(T) + a[:,3] * T + a[:,4] * T^2 /2 + a[:,5] * T^3 /3 + a[:,6] * T^4 / 4 + a[:,8]) * R
    return [v1 v2 v3]
end
 
function f_lo(a, T, R)
    v1 = (a[:,9] + a[:,10] * T + a[:,11] * T^2 + a[:,12] * T^3 + a[:,13] * T^4) * R
    v2 = (a[:,9] + a[:,10] * T / 2 + a[:,11] * T^2 /3 + a[:,12] * T^3 /4 + a[:,13] * T^4 /5 + a[:,14]/T) * R * T
    v3 = (a[:,9] * log(T) + a[:,10] * T + a[:,11] * T^2 /2 + a[:,12] * T^3 /3 + a[:,13] * T^4 / 4 + a[:,15]) * R
    return [v1 v2 v3]
end 

function compute_falloff(T, pr, a)
    fcent = (1 - a[1]) * exp(-T/a[2]) + a[1] * exp(-T/a[3]) + exp(-a[4]/T)
    c = -0.4 - 0.67 * log(fcent) / log(10)
    n = 0.75 - 1.27 * log(fcent) / log(10)
    f1 = (log(pr) / log(10) + c) / (n - 0.14 *(log(pr) / log(10) + c))
    return  10 ^ ((log(fcent) / log(10)) / (1 + f1 ^ 2))
end

compute_falloff (generic function with 1 method)

In [3]:
V = 1  # Volume
m = (py"np.load"("data/density.npy"))[1]  # Total mass
tbd = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/tbd.npy") .+ 1
falofr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/falofr.npy") .+ 1
elmr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/elmr.npy") .+ 1
NASA_coeffs = py"np.load"("data/NASA_coeffs.npy")
W = py"np.load"("data/molecular_weights.npy") # Molar weight
ν1 = py"np.load"("data/reactants_stoich_coeffs.npy") # Forward molar stoichiometric coefficients
ν2 = py"np.load"("data/product_stoich_coeffs.npy") # Backward model stoichiometric coefficients
reversible = py"np.load"("data/reversible.npy")
N = size(ν1)[1]  # Number of Species
M = size(ν1)[2]  # Number of Reactions
ν1_order = zeros(N,M)
ν = ν2 - ν1  
pa = 100000 # 1 bar
R = 8314.4621 # Gas constant in kmol
### Constants: Combustion Chamber Level
min_dot = 0 # Rate at which mass enters the chamber
mout_dot = 0 # Rate at which mass leaves the chamber
Yin = ones(N) # Mass fraction of species entering the chamber
Yout = ones(N) # Mass fraction of species leaving the chamber
Qdot = 0 # Heating source
m_dot = min_dot - mout_dot
hin = 1 # Enthalpy of input
### Unknowns
Tt_cant = py"np.load"("data/temperature.npy")
T = Tt_cant[1] # Temperature
Y = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/ini_mass_fraction.npy") # Mass fractions
#Y .+= 0.00001
#Y ./ sum(Y)
### Quantities that depend on the Unknowns
tbd = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/tbd.npy") .+ 1
tbd = Int.(tbd)
falofr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/falofr.npy") .+ 1
falofr = Int.(falofr)
elmr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/elmr.npy") .+ 1
elmr = Int.(elmr)

order = py"np.load"("data/reaction_orders.npy")
Af = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor.npy") # preexponential constant Afj
β = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent.npy") # Temperature exponent
E = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy.npy") # Activation energy for the reactions in kJ

order_t = py"np.load"("data/reaction_orders_t.npy")
efficiency_t = py"np.load"("data/efficiency_t.npy")
Af_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_t.npy") # preexponential constant Afj
β_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_t.npy") # Temperature exponent
E_t = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_t.npy") # Activation energy for the reactions in kJ

order_f = py"np.load"("data/reaction_orders_f.npy")
troefall = py"np.load"("data/troefall.npy") .+ 1
troefall_coeff = py"np.load"("data/troefall_coeff.npy")
efficiency_f = py"np.load"("data/efficiency_f.npy")
Af_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_hi.npy") # preexponential constant Afj
β_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_hi.npy") # Temperature exponent
E_hi = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_hi.npy") 

Af_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor_lo.npy") # preexponential constant Afj
β_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent_lo.npy") # Temperature exponent
E_lo = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy_lo.npy") 

ν1_order[:,elmr] = order
ν1_order[:,tbd] = order_t
ν1_order[:,falofr] = order_f;

In [4]:
function q(t)
    t0 = 1e-5
    r = 1e-6
    c = 1e12
    return c * exp(-(t-t0)^2 / r^2)
end
function f(TY, Qdot, t)
    T = TY[1]
    Y = TY[2:end]
    ρ = m / V # density
    X = tf.reshape(ρ * tf.divide(Y, W), (N,1)) # Concentration
    Q = ADCME.constant(ones(M)) # Individual progress rates
    islarge = T > ADCME.constant(NASA_coeffs[:,1])
    islarge = convert_to_tensor(islarge, dtype=Float64)
    fhi = f_hi(NASA_coeffs, T, R)
    flo = f_lo(NASA_coeffs, T, R)
    cp = islarge .* fhi[:,1] + (1 - islarge) .* flo[:,1]
    h = islarge .* fhi[:,2] + (1 - islarge) .* flo[:,2]
    s = islarge .* fhi[:,3] + (1 - islarge) .* flo[:,3]
    cvk = cp .- R
    ΔS = ν' * s  # Entropy change for reaction j
    ΔH = ν' * h # Entahlpy change for reaction j
    ####
    M_t = efficiency_t' * X[:,1]
    Kf_t = Af_t .* (T ^ β_t) .* exp(-E_t / (R * T)) .* M_t
    Kr_t = Kf_t ./ (((pa/(R * T)) ^ sum(ν[:,tbd], dims=1)')[:,1] .* exp(ΔS[tbd] ./ R - ΔH[tbd] / (R * T)))
    Qtbd =  Kf_t .* (prod(tf.pow(X, order_t), dims=1)) .- Kr_t .* (prod(tf.pow(X, ν2[:,tbd]), dims=1) .* reversible[tbd])
    M_f = efficiency_f' * X[:,1]
    Kf_lo = Af_lo .* (T ^ β_lo) .* exp(-E_lo / (R * T)) .* M_f
    Kf_hi = Af_hi .* (T ^ β_hi) .* exp(-E_hi / (R * T)) 
    Pr = Kf_lo ./ Kf_hi
    Fac = ADCME.constant(ones(size(falofr)[1]))
    for (i,s) in enumerate(troefall)
        a = troefall_coeff[:,i]
        fcent = (1 - a[1]) * exp(-T/a[2]) + a[1] * exp(-T/a[3]) + exp(-a[4]/T)
        c = -0.4 - 0.67 * log(fcent) / log(10)
        n = 0.75 - 1.27 * log(fcent) / log(10)
        f1 = (log(Pr[s]) / log(10) + c) / (n - 0.14 *(log(Pr[s]) / log(10) + c))
        Fac = scatter_update(Fac, troefall[i], 10 ^ ((log(fcent) / log(10)) / (1 + f1 ^ 2)))
    end
    Kf_f = Kf_lo ./ (1 .+ (Kf_lo ./ Kf_hi)) .* Fac
    Kr_f = Kf_f ./ (((pa/(R * T)) ^ sum(ν[:,falofr], dims=1)')[:,1] .* exp(ΔS[falofr] ./ R - ΔH[falofr] / (R * T)))
    Qfalofr = Kf_f .* (prod(tf.pow(X, order_f), dims=1)) .- Kr_f .* (prod(tf.pow(X, ν2[:,falofr]), dims=1) .* reversible[falofr])
    ####
#     Af = exp(θ[:,1])
#     β = zeros(M)
#     E = exp(θ[:,3])
    Kf = Af .* (T ^ β) .* exp(-E / (R * T))
    Kr = Kf ./ (((pa/(R * T)) ^ sum(ν[:,elmr], dims=1)')[:,1] .* exp(ΔS[elmr] ./ R - ΔH[elmr] / (R * T)))
    Qelmr = Kf .* (prod(tf.pow(X, order), dims=1)) .- Kr .* (prod(tf.pow(X, ν2[:,elmr]), dims=1) .* reversible[elmr])
    Q = [Qtbd' Qfalofr' Qelmr']
    ν_new = [ν[:,tbd] ν[:,falofr] ν[:,elmr]]
    ##### Computing ω_dot 
    cv = sum(cvk ./ W .* Y) # Mass heat capacities
    u = h / W - R ./ W * T   # Internal energy for species
    p = sum(X) * R * T # pressure
    ω_dot = W .* sum(ν_new .* Q, dims=2)
    ###### Species Conservation
    mgen_dot = V * ω_dot
    Y_dot = (1 / m) * ((min_dot * (Yin - Y) - mout_dot * Y) + mgen_dot) 
    ###### EnergyConservation
    #Qdot = abs(fc(t * 1e5, [20,20,1], theta[2]) + 1) * 10 ^ theta[1] 
    #Qdot = theta[1] * 1e11 * exp(-(t * 1e5 - theta[2]) ^ 2 / (0.01))
    T_dot = 1 / (m * cv) * (Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))
    return tf.concat([tf.reshape(T_dot, (1,)), Y_dot], 0)
end

f (generic function with 1 method)

In [5]:
timestep = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/timestep.npy")
Tt_cant = py"np.load"("data/temperature.npy")
T_ref = zeros(size(timestep)[1])
T_ref[2:end] = Tt_cant
T_ref[1] = 500
Yt_cant = py"np.load"("data/mass_fraction.npy")
Y_ref = [Y Yt_cant]
obs = [T_ref Y_ref']

2001×7 Array{Float64,2}:
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0   0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
    ⋮        

In [6]:
Afr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/pre_exponential_factor.npy") # preexponential constant Afj
βr = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/temperature_exponent.npy") # Temperature exponent
Er = py"np.load"("/home/darve/adncat/yizhou/PSAAP3/Tests/data/activation_energy.npy") # Activation energy for the reactions in kJ
pr = [log.(Afr) βr log.(Er)]
pr[1,1] = 28.25
pr[2,1] = 17.75
pr[1,3] = 18.75
pr[2,3] = 17.75

17.75

In [7]:
nt = size(timestep)[1]
inds = range(1,2000, step=50)
TY = [T; Y]
function condition(k, U, t)
    k <= nt - 1
end

function body(k, U, t)
    dt = t[k + 1] - t[k]
    uk = read(U,k)
    k1 = f(uk, Qdot[k], t[k])
    k2 = f(uk + 0.5 * dt * k1, Qdot[k], t[k] + 0.5 * dt)
    k3 = f(uk + 0.5 * dt * k2, Qdot[k], t[k] + 0.5 * dt)
    k4 = f(uk + dt * k3, Qdot[k], t[k] + dt) 
    u_new = uk + 1/6 * dt * (k1 + 2 * k2 + 2 * k3 + k4)
    U = write(U, k+1, u_new)
    k+1, U, t
end
k = constant(1, dtype=Int32)
t = constant(timestep)
θ = Variable(fc_init([1,20,20,1]))
C = Variable(3.0)
#theta = Variable([8.0, 0.8])
Qdtb = abs(fc(t * 1e5, [20,20,1], θ))
Qdot = 1.8 * (1.0 / (1.0 + exp(-C))) * 1e6 * Qdtb / sum(Qdtb[1:end-1] * (t[2:end] - t[1:end-1]))
U = TensorArray(nt)
U = write(U, 1, TY)
_, U_out=while_loop(condition, body, [k, U, t])
U_array = set_shape(stack(U_out), (nt, N+1))
loss = sum((U_array[inds,1]-obs[inds,1])^2)
sess = Session()
init(sess)
#u_out = run(sess, U_array)
loss_hist = BFGS!(sess, loss, 750)
#print(loss_hist)
out = run(sess, U_array)

2020-11-29 18:31:52.154831: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX
2020-11-29 18:31:52.168361: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2600020000 Hz
2020-11-29 18:31:52.173179: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x9660ba0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-29 18:31:52.173249: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-11-29 18:31:52.182905: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/darve/adncat/.conda/envs/my_root/lib/libfabric:/home/darve/adncat/.conda/envs/my_root/lib:/opt/ohpc/pub/compiler/gcc/7.3.0/lib64:/opt/ohpc/pu

iter 0, current loss=6.646854521041674e6
iter 1, current loss=3.381582029827431e7
iter 2, current loss=5.159849989300523e6
================ STEP 0 ===============
iter 3, current loss=6.736370998417625e6
iter 4, current loss=4.959655856624292e6
================ STEP 1 ===============
iter 5, current loss=1.1668425327305902e7
iter 6, current loss=4.896404733653325e6
================ STEP 2 ===============
iter 7, current loss=8.677565506217524e6
iter 8, current loss=4.840176520436827e6
================ STEP 3 ===============
iter 9, current loss=4.767682301942314e6
iter 10, current loss=4.365517560201789e6
iter 11, current loss=1.1874225984513797e7
iter 12, current loss=4.327434192118552e6
iter 13, current loss=2.2295885968180854e7
iter 14, current loss=4.1730791211482394e6
iter 15, current loss=4.216649727657601e6
iter 16, current loss=4.0606183441615077e6
================ STEP 4 ===============
iter 17, current loss=3.0722159615308635e7
iter 18, current loss=4.057888656048855e6
iter 1

iter 119, current loss=1.975523081006335e6
================ STEP 78 ===============
iter 120, current loss=1.9708857159141912e6
================ STEP 79 ===============
iter 121, current loss=1.9674784250350823e6
================ STEP 80 ===============
iter 122, current loss=1.9649375588754653e6
================ STEP 81 ===============
iter 123, current loss=1.9627739214461925e6
================ STEP 82 ===============
iter 124, current loss=1.9597764900807333e6
================ STEP 83 ===============
iter 125, current loss=1.9582794455834744e6
================ STEP 84 ===============
iter 126, current loss=1.9551768229408674e6
================ STEP 85 ===============
iter 127, current loss=1.9405342826720076e6
================ STEP 86 ===============
iter 128, current loss=2.02278605763837e6
iter 129, current loss=1.9304197896954368e6
================ STEP 87 ===============
iter 130, current loss=2.364388715607216e6
iter 131, current loss=1.9236606658799727e6
================ STEP 

iter 228, current loss=765869.792321787
================ STEP 164 ===============
iter 229, current loss=765743.5219663468
================ STEP 165 ===============
iter 230, current loss=765448.3328523191
================ STEP 166 ===============
iter 231, current loss=764165.9350257475
================ STEP 167 ===============
iter 232, current loss=761641.8879622268
================ STEP 168 ===============
iter 233, current loss=757696.5992919835
================ STEP 169 ===============
iter 234, current loss=813665.8186894553
iter 235, current loss=757177.8610066529
================ STEP 170 ===============
iter 236, current loss=755253.4426629582
================ STEP 171 ===============
iter 237, current loss=754387.0856426083
================ STEP 172 ===============
iter 238, current loss=753870.877126585
================ STEP 173 ===============
iter 239, current loss=753655.832570117
================ STEP 174 ===============
iter 240, current loss=750893.4490212351
========

iter 337, current loss=313436.90217876656
iter 338, current loss=290381.82427533367
================ STEP 252 ===============
iter 339, current loss=280120.8248849994
================ STEP 253 ===============
iter 340, current loss=269713.24651526066
================ STEP 254 ===============
iter 341, current loss=247196.0970862512
================ STEP 255 ===============
iter 342, current loss=243216.91439787028
iter 343, current loss=232089.03438278247
================ STEP 256 ===============
iter 344, current loss=1.1744380615411736e6
iter 345, current loss=226777.23523767496
iter 346, current loss=216848.05192324205
================ STEP 257 ===============
iter 347, current loss=206683.76779383013
================ STEP 258 ===============
iter 348, current loss=198701.27550432686
================ STEP 259 ===============
iter 349, current loss=189571.29002014198
================ STEP 260 ===============
iter 350, current loss=171248.41530256261
================ STEP 261 ========

...

iter 549, current loss=17940.201576832576
================ STEP 435 ===============
iter 550, current loss=17641.69556392952
================ STEP 436 ===============
iter 551, current loss=17463.1439552504
================ STEP 437 ===============
iter 552, current loss=17268.400839356764
================ STEP 438 ===============
iter 553, current loss=18336.776556930028
iter 554, current loss=17240.528733891177
================ STEP 439 ===============
iter 555, current loss=17465.655602261264
iter 556, current loss=17128.41611938893
================ STEP 440 ===============
iter 557, current loss=16935.346204329708
================ STEP 441 ===============
iter 558, current loss=16719.860159520787
================ STEP 442 ===============
iter 559, current loss=16400.322285188817
================ STEP 443 ===============
iter 560, current loss=15669.187632441775
================ STEP 444 ===============
iter 561, current loss=35078.7774239179
iter 562, current loss=15311.32365358150

iter 659, current loss=963.8527285866201
================ STEP 523 ===============
iter 660, current loss=925.2242756100999
================ STEP 524 ===============
iter 661, current loss=881.4810332559061
================ STEP 525 ===============
iter 662, current loss=872.5509809914956
================ STEP 526 ===============
iter 663, current loss=853.7975006152549
================ STEP 527 ===============
iter 664, current loss=835.1842186617567
================ STEP 528 ===============
iter 665, current loss=828.810332410102
================ STEP 529 ===============
iter 666, current loss=820.1636414910686
================ STEP 530 ===============
iter 667, current loss=817.2912669021409
================ STEP 531 ===============
iter 668, current loss=810.755703070251
================ STEP 532 ===============
iter 669, current loss=809.9238037294804
================ STEP 533 ===============
iter 670, current loss=806.4019379415874
================ STEP 534 ===============
iter 6

iter 764, current loss=677.065277142462
iter 765, current loss=671.1009401426031
================ STEP 616 ===============
iter 766, current loss=667.8837380591254
================ STEP 617 ===============
iter 767, current loss=665.1287328672311
================ STEP 618 ===============
iter 768, current loss=662.3817038689326
================ STEP 619 ===============
iter 769, current loss=662.2006660061045
================ STEP 620 ===============
iter 770, current loss=661.1336721133874
================ STEP 621 ===============
iter 771, current loss=660.9805337970255
================ STEP 622 ===============
iter 772, current loss=660.8300771936347
================ STEP 623 ===============
iter 773, current loss=660.6611907411555
================ STEP 624 ===============
iter 774, current loss=660.5713929464963
================ STEP 625 ===============
iter 775, current loss=660.6145536719661
iter 776, current loss=660.4708852291419
================ STEP 626 ===============
iter 7

iter 870, current loss=513.8292377844634
================ STEP 709 ===============
iter 871, current loss=513.0134552707118
================ STEP 710 ===============
iter 872, current loss=512.187908091729
================ STEP 711 ===============
iter 873, current loss=512.2156938763678
iter 874, current loss=511.6386554449605
================ STEP 712 ===============
iter 875, current loss=511.96230404883966
iter 876, current loss=511.33467000999553
================ STEP 713 ===============
iter 877, current loss=527.8184280576995
iter 878, current loss=510.8218197103181
================ STEP 714 ===============
iter 879, current loss=510.3027942256889
================ STEP 715 ===============
iter 880, current loss=508.91122728109076
================ STEP 716 ===============
iter 881, current loss=507.6370053300295
================ STEP 717 ===============
iter 882, current loss=504.42065867310214
================ STEP 718 ===============
iter 883, current loss=498.5409526754056
===

2001×7 Array{Float64,2}:
  500.0    0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0    0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0    0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0    0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0    0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0    0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0    0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0    0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0    0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0    0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.0    0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.001  0.166667     0.166667  0.166667   0.166667  0.166667   0.166667
  500.001  0.166667     0.166667  0.166667   0.166667  0.166667   0.166667


In [8]:
loss_hist2 = BFGS!(sess, loss, 250)

┌ Info: Optimization starts...
└ @ ADCME /home/darve/adncat/.julia/packages/ADCME/OGm7w/src/optim.jl:326


iter 0, current loss=471.5271717215378
iter 1, current loss=1.8727727458477274e7
iter 2, current loss=471.75988977897754
iter 3, current loss=471.51123125567636
================ STEP 0 ===============
iter 4, current loss=471.50357871372
================ STEP 1 ===============
iter 5, current loss=471.49218760951334
================ STEP 2 ===============
iter 6, current loss=471.44774845444243
================ STEP 3 ===============
iter 7, current loss=471.3866401099938
================ STEP 4 ===============
iter 8, current loss=471.24907488916074
================ STEP 5 ===============
iter 9, current loss=471.14839047551715
================ STEP 6 ===============
iter 10, current loss=471.0950690458585
================ STEP 7 ===============
iter 11, current loss=471.07581085213417
================ STEP 8 ===============
iter 12, current loss=471.0481269751032
================ STEP 9 ===============
iter 13, current loss=470.94471781477046
================ STEP 10 ===============


iter 106, current loss=408.2225502433315
================ STEP 97 ===============
iter 107, current loss=407.1338494084465
================ STEP 98 ===============
iter 108, current loss=406.8954699970433
================ STEP 99 ===============
iter 109, current loss=408.1966754547483
iter 110, current loss=406.4232595263354
================ STEP 100 ===============
iter 111, current loss=406.0039269746794
================ STEP 101 ===============
iter 112, current loss=403.8673816688836
================ STEP 102 ===============
iter 113, current loss=402.5829395590048
================ STEP 103 ===============
iter 114, current loss=400.24549097216584
================ STEP 104 ===============
iter 115, current loss=397.8147852103431
================ STEP 105 ===============
iter 116, current loss=396.78069737505444
================ STEP 106 ===============
iter 117, current loss=394.4895911118918
================ STEP 107 ===============
iter 118, current loss=392.85563185601126
=====

iter 208, current loss=306.9480688029805
================ STEP 192 ===============
iter 209, current loss=302.4300173370096
================ STEP 193 ===============
iter 210, current loss=305.7218167058582
iter 211, current loss=300.5811713047591
================ STEP 194 ===============
iter 212, current loss=297.8096571219171
================ STEP 195 ===============
iter 213, current loss=347.33123528836137
iter 214, current loss=297.13499040979406
================ STEP 196 ===============
iter 215, current loss=296.51037833329417
================ STEP 197 ===============
iter 216, current loss=294.8670467269279
================ STEP 198 ===============
iter 217, current loss=297.93072354019773
iter 218, current loss=294.68184711977835
================ STEP 199 ===============
iter 219, current loss=294.35445959078146
================ STEP 200 ===============
iter 220, current loss=294.0095996251415
================ STEP 201 ===============
iter 221, current loss=293.6904658108267


250-element Array{Any,1}:
 471.51123125567636
 471.50357871372
 471.49218760951334
 471.44774845444243
 471.3866401099938
 471.24907488916074
 471.14839047551715
 471.0950690458585
 471.07581085213417
 471.0481269751032
 470.94471781477046
 470.80174800494353
 470.70237848874126
   ⋮
 291.1215016392189
 291.054952492531
 291.03405059496373
 290.99610822778254
 290.87534786089833
 290.72164273985396
 290.3441041455093
 289.9763612666881
 289.7289876137967
 289.550097931315
 289.40751045192695
 289.3332231535807

In [ ]:
###### nt = size(timestep)[1]
TY = [T; Y]
function condition(k, U, t, θ)
    k <= nt - 1
end

function body(k, U, t, θ)
    dt = t[k + 1] - t[k]
    uk = read(U,k)
    k1 = f(uk, θ)
    k2 = f(uk + 0.5 * dt * k1, θ)
    k3 = f(uk + 0.5 * dt * k2, θ)
    k4 = f(uk + dt * k3, θ) 
    u_new = uk + 1/6 * dt * (k1 + 2 * k2 + 2 * k3 + k4)
    U = write(U, k+1, u_new)
    k+1, U, t, θ
end
k = constant(1, dtype=Int32)
t = constant(timestep)
θ = Variable(pr)
U = TensorArray(nt)
U = write(U, 1, TY)
_, U_out=while_loop(condition, body, [k, U, t, θ])
U_array = set_shape(stack(U_out), (nt, N+1))
sess = Session()
init(sess)
u_out = run(sess, U_array)

In [ ]:
#dt = 1e-6
nt = size(timestep[1:1000])[1]
#### Implicit scheme, need to refine for variable time step and order
function res_and_jac(param, x)
    dt = param[1]
    x0 = param[2:end]
    res = x - x0 - dt * f(x)
    jac = gradients(res, x)
    res, jac 
end

function condition(k, U, t)
    k <= nt - 1
end

function body(k, U, t)
    dt = t[k + 1] - t[k]
    uk = read(U,k)
    ADCME.options.newton_raphson.rtol = 1e-5 # relative tolerance
    ADCME.options.newton_raphson.tol = 1e-5 # absolute tolerance
    ADCME.options.newton_raphson.verbose = true # print details in newton_raphson
    param = tf.concat([tf.reshape(dt, (1,)), uk], 0)
    u_new = newton_raphson_with_grad(res_and_jac, constant(uk), param)
    U = write(U, k+1, u_new)
    k+1, U, t
end

k = constant(1, dtype=Int32)
t = constant(timestep)
U = TensorArray(nt)
U = write(U, 1, TY)
_, U_out=while_loop(condition, body, [k, U, t])
U_array = set_shape(stack(U_out), (nt, N+1))
sess = Session()
init(sess)
u_out = run(sess, U_array)

In [ ]:
#inds = range(1,2000, step=100)
plot(timestep[inds], Y_ref'[inds,:], seriestype = :scatter, legend = false)
plot!(timestep, u_out[:,2:end])
xlabel!("Time")
ylabel!("Mass Fractions")
savefig("Mass_fraction.pdf")

In [20]:
#inds = range(1,2000, step=400)
plot(timestep[inds], T_ref[inds], seriestype = :scatter, label = "observation")
plot!(timestep[2:end], Tt_cant, label = "true")
plot!(timestep, out[:,1], label = "predicted")
xlabel!("Time")
ylabel!("Temperature")
savefig("temperature_3.pdf")

In [ ]:
dt = 1e-6
u1 = constant(TY)
function res_and_jac(x0, x)
    res = x - x0 - dt * f(x)
    jac = gradients(res, x)
    res, jac 
end
ADCME.options.newton_raphson.rtol = 1e-4 # relative tolerance
ADCME.options.newton_raphson.tol = 1e-4 # absolute tolerance
ADCME.options.newton_raphson.verbose = true # print details in newton_raphson
u_est = newton_raphson_with_grad(res_and_jac, constant(u1), u1)

In [ ]:
sess = Session()
init(sess)
u_e = run(sess, u_est)

In [ ]:
28.3244  0.0  18.8021
 17.9616  0.0  17.7315

In [ ]:
u_out[range(1,2000, step=100),2:end]

In [ ]:
pr = [log.(Afr) βr log.(Er)]

In [ ]:
loss

In [ ]:
plot(log.(loss), label="log of mse")
xlabel!("steps")
ylabel!("logarithm of loss")
savefig("loss.pdf")

In [ ]:
θ = Variable(fc_init([1,20,20,1]))

In [ ]:
XX = Variable(0.5)
fc(XX, [20,20,1], θ)

In [ ]:
ty = Variable(TY)

In [ ]:
f(ty, θ, constant(0.1))

In [ ]:
tout_g = q.(timestep)

In [ ]:
tout = fc(timestep, [20,20,1], θ)
loss = sum((tout - tout_g)^2)

In [ ]:
#θ = Variable(fc_init([1,20,20,1]))
#C = Variable(11.0)
tout = fc(timestep * 1e5, [20,20,1], θ; activation = "tanh")
pred = tout * 10 ^ C
#loss = sum((pred - tout_g)^2) 
#sess = Session()
#init(sess)
#u_out = run(sess, U_array)
#loss_hist = BFGS!(sess, loss)
tout = run(sess, pred)

In [ ]:
#plot(timestep, tout_g)
plot(timestep, tout)

In [10]:
function q(t)
    t0 = 1
    r = 0.1
    c = 1e12
    return c * exp(-(t-t0)^2 / r^2)
end
tout_g = q.(timestep * 1e5)

2001-element Array{Float64,1}:
 3.7200759760208887e-32
 3.7208200652505616e-32
 3.7223086880040565e-32
 3.7245427333101807e-32
 3.727523535812236e-32
 3.731252877090545e-32
 3.7357329874283037e-32
 3.7409665480229064e-32
 3.746956693647616e-32
 3.753707015764383e-32
 3.761221566095272e-32
 3.769504860654328e-32
 3.778561884248479e-32
 ⋮
 2.348330389857639e-30
 1.5898579986997044e-30
 1.0752962189675263e-30
 7.265542158660972e-31
 4.904305803407465e-31
 3.30716714742584e-31
 2.227939102192905e-31
 1.4994035491696873e-31
 1.008094583257523e-31
 6.770972273722614e-32
 4.543256709289263e-32
 3.045436046522662e-32

In [ ]:
out_C = run(sess, C)

In [ ]:
out_theta = run(sess, θ)

In [ ]:
run(sess, C)

In [ ]:
run(sess, θ)

In [ ]:
fc(timestep * 1e5, [20,20,1], θ; activation = "tanh")

In [ ]:
ty = Variable(TY)

In [ ]:
fc(t * 1e5, [20,20,1], temp)

In [ ]:
theta[2]

In [ ]:
temp = Variable(fc_init([1,20,20,1]))

In [ ]:
[C, θ]

In [ ]:
t

In [ ]:
out_C

In [ ]:
out_theta

In [ ]:
loss_hist

In [ ]:
fc(t * 1e5, [20,20,1], theta[2]; activation = "tanh") 

In [ ]:
t = constant(0.01)
T = ty[1]
Y = ty[2:end]
ρ = m / V # density
X = tf.reshape(ρ * tf.divide(Y, W), (N,1)) # Concentration
Q = ADCME.constant(ones(M)) # Individual progress rates
islarge = T > ADCME.constant(NASA_coeffs[:,1])
islarge = convert_to_tensor(islarge, dtype=Float64)
fhi = f_hi(NASA_coeffs, T, R)
flo = f_lo(NASA_coeffs, T, R)
cp = islarge .* fhi[:,1] + (1 - islarge) .* flo[:,1]
h = islarge .* fhi[:,2] + (1 - islarge) .* flo[:,2]
s = islarge .* fhi[:,3] + (1 - islarge) .* flo[:,3]
cvk = cp .- R
ΔS = ν' * s  # Entropy change for reaction j
ΔH = ν' * h # Entahlpy change for reaction j
####
M_t = efficiency_t' * X[:,1]
Kf_t = Af_t .* (T ^ β_t) .* exp(-E_t / (R * T)) .* M_t
Kr_t = Kf_t ./ (((pa/(R * T)) ^ sum(ν[:,tbd], dims=1)')[:,1] .* exp(ΔS[tbd] ./ R - ΔH[tbd] / (R * T)))
Qtbd =  Kf_t .* (prod(tf.pow(X, order_t), dims=1)) .- Kr_t .* (prod(tf.pow(X, ν2[:,tbd]), dims=1) .* reversible[tbd])
M_f = efficiency_f' * X[:,1]
Kf_lo = Af_lo .* (T ^ β_lo) .* exp(-E_lo / (R * T)) .* M_f
Kf_hi = Af_hi .* (T ^ β_hi) .* exp(-E_hi / (R * T)) 
Pr = Kf_lo ./ Kf_hi
Fac = ADCME.constant(ones(size(falofr)[1]))
for (i,s) in enumerate(troefall)
    a = troefall_coeff[:,i]
    fcent = (1 - a[1]) * exp(-T/a[2]) + a[1] * exp(-T/a[3]) + exp(-a[4]/T)
    c = -0.4 - 0.67 * log(fcent) / log(10)
    n = 0.75 - 1.27 * log(fcent) / log(10)
    f1 = (log(Pr[s]) / log(10) + c) / (n - 0.14 *(log(Pr[s]) / log(10) + c))
    Fac = scatter_update(Fac, troefall[i], 10 ^ ((log(fcent) / log(10)) / (1 + f1 ^ 2)))
end
Kf_f = Kf_lo ./ (1 .+ (Kf_lo ./ Kf_hi)) .* Fac
Kr_f = Kf_f ./ (((pa/(R * T)) ^ sum(ν[:,falofr], dims=1)')[:,1] .* exp(ΔS[falofr] ./ R - ΔH[falofr] / (R * T)))
Qfalofr = Kf_f .* (prod(tf.pow(X, order_f), dims=1)) .- Kr_f .* (prod(tf.pow(X, ν2[:,falofr]), dims=1) .* reversible[falofr])
####
#     Af = exp(θ[:,1])
#     β = zeros(M)
#     E = exp(θ[:,3])
Kf = Af .* (T ^ β) .* exp(-E / (R * T))
Kr = Kf ./ (((pa/(R * T)) ^ sum(ν[:,elmr], dims=1)')[:,1] .* exp(ΔS[elmr] ./ R - ΔH[elmr] / (R * T)))
Qelmr = Kf .* (prod(tf.pow(X, order), dims=1)) .- Kr .* (prod(tf.pow(X, ν2[:,elmr]), dims=1) .* reversible[elmr])
Q = [Qtbd' Qfalofr' Qelmr']
ν_new = [ν[:,tbd] ν[:,falofr] ν[:,elmr]]
##### Computing ω_dot 
cv = sum(cvk ./ W .* Y) # Mass heat capacities
u = h / W - R ./ W * T   # Internal energy for species
p = sum(X) * R * T # pressure
ω_dot = W .* sum(ν_new .* Q, dims=2)
###### Species Conservation
mgen_dot = V * ω_dot
Y_dot = (1 / m) * ((min_dot * (Yin - Y) - mout_dot * Y) + mgen_dot) 
###### EnergyConservation
Qdot = fc(t * 1e5, [20,20,1], θ) 
T_dot = 1 / (m * cv) * (Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))

In [ ]:
tout = fc(timestep * 1e5, [20,20,1], θ; activation = "tanh")
pred = tout * 10 ^ C
out = run(sess, pred)

In [ ]:
out = run(sess, loss)

In [ ]:
run(sess, loss)

In [ ]:
sum((u_out[inds,1]-obs[inds,1]).^2)

In [ ]:
loss = sum((U_array[inds,1]-obs[inds,1])^2)

In [ ]:
Tt_cant

In [ ]:
u_out = run(sess, U_array)

In [ ]:
u_out

In [ ]:
run(sess, theta)

In [ ]:
Variable([0.0, 0.0])

In [ ]:
plot(log.(loss_hist))

In [ ]:
argmax(Tt)

In [ ]:
timestep[argmax(Tt[1:20000])]

In [ ]:
out

In [ ]:
nt

In [ ]:
total_heat = 0
for i = 1:nt-1
    dt = timestep[i+1] - timestep[i]
    total_heat += dt * q(timestep[i])
end

In [ ]:
has_gpu()

In [ ]:
total_heat

In [ ]:
θ = Variable(fc_init([1,20,20,1]))
#C = Variable(1.0)
#theta = Variable([8.0, 0.8])
Qdtb = fc(t * 1e5, [20,20,1], θ)

In [ ]:
sum(Qdtb)

In [ ]:
delta_t = t[2:end] - t[1:end-1]

In [ ]:
sum(Qdtb[1:end-1] * (t[2:end] - t[1:end-1]))

In [9]:
Qdot_out = run(sess, Qdot)

2001-element Array{Float64,1}:
 4.698905719255907e8
 4.6985093654704016e8
 4.6977166938196117e8
 4.696527776132711e8
 4.6949427201693803e8
 4.692961669589928e8
 4.690584803993005e8
 4.68781233887985e8
 4.6846445256792694e8
 4.681081651743228e8
 4.6771240403424424e8
 4.672772050680586e8
 4.6680260778648967e8
 ⋮
 1.259904077679363e10
 1.266829638586406e10
 1.2737630979728926e10
 1.2807043767034134e10
 1.2876533952951967e10
 1.2946100739224539e10
 1.301574332420866e10
 1.3085460902918314e10
 1.3155252667070118e10
 1.322511780512863e10
 1.3295055502352098e10
 1.3365064940837793e10

In [14]:
plot(timestep, Qdot_out, label="Neural Network")
plot!(timestep, tout_g, label="Gaussian Heat Source")
xlabel!("Time (s)")
ylabel!("Heat Flux (J)")
savefig("heat_source_3.pdf")

In [18]:
plot([loss_hist; loss_hist2], yaxis=:log, label = "BFGS")
xlabel!("steps")
ylabel!("loss")
savefig("loss_3.pdf")

In [ ]:
plot((Qdot_out .- tout_g) ./ maximum(tout_g))

In [ ]:
out

In [ ]:
tout_g

In [ ]:
ty = Variable(TY)

In [ ]:
f(ty, Qdot, 0.1)

In [ ]:
using Pkg
ENV["GPU"] = 0
Pkg.build("ADCME")

In [ ]:
using ADCME

In [ ]:
has_gpu()

In [ ]:
out = run(sess, U_array)